# Lexicon Creation

In [ ]:
#pip install symspellpy
from pyspark.sql import SparkSession
from pyspark.sql.functions import split
from PyClasses.Scrapers.PRPMScraper import PRPMScraper
from PyClasses.Preprocessing.PRPMCleaner import PRPMCleaner
from datetime import datetime, timedelta

import malaya.stem

class LexiconBuilder:
    def __init__(self, spark):
        self.spark = spark
        
    def get_reduced_words(self):
        try:
            file_path = "DE-prj/MR_WC_Result/part-00000"
            df = self.spark.read.text(file_path)
            df_split = df.select(split(df['value'], '\t').alias('word_count'))
            df_final = df_split.select(
                df_split['word_count'].getItem(0).alias('word'),  # First item is the word
                df_split['word_count'].getItem(1).cast('int').alias('count')  # Second item is the count, cast it to integer
            )
            return df_final
        except:
            return None


    def get_words_properties(self):
        df = self.get_reduced_words()
        PRPMscrap = PRPMScraper()
        modelBC = spark.sparkContext.broadcast(malaya.stem.huggingface())
        def get_word_metadata(rows):
            result = []
            PRPMscrap = PRPMScraper()
            model = modelBC.value
            for row in rows:
                singleResult = PRPMscrap.findWordMetaData(row.word)
                if singleResult is not None:
                    singleResult[row.word]["base"] = model.stem(row.word)
                    singleResult[row.word]["count"] = row["count"]
                    print(f'Ori:{row.word}, base:{singleResult[row.word]["base"]}, count:{singleResult[row.word]["count"]}')
                    result.append(singleResult)
            return result
        def get_word_metadata_single(row):
            print(row.word)
            return broadCastPRPMScrap_obj.value.findWordMetaData(row.word)
        #find all words meaning from prpm
        #wordRows_rdd = self.spark.sparkContext.parallelize(df)
        df.show()
        wordRows_rdd = df.rdd.repartition(4)
        timeNow = datetime.now() 
        wordRows_rdd.foreachPartition(get_word_metadata)
        timeEnd =  datetime.now() 
        timeTaken = timeEnd - timeNow
        print(f'Processing Start Time: {timeNow}')
        print(f'Processing End Time: {timeEnd}')
        print(f'Total Time Taken: {timeTaken}')
        return 'success'
        
spark = SparkSession.builder.appName("DE-prj").getOrCreate()
lexiconBuilder = LexiconBuilder(spark)
try:
    lexiconBuilder.get_words_properties()
finally:
    pass
    # Word, wordcount, synonym, antonym, definition, stemmed

+-----------+-----+
|       word|count|
+-----------+-----+
|         ab|    3|
|       abai|    1|
|        abc|    1|
|        abd|    1|
|      abdul|   19|
|    abiddin|    1|
|      abong|    1|
|        abu|    2|
|         ac|    4|
|      acara|    5|
|accelerator|    2|
|         ad|    1|
|        ada|   52|
|     adakan|    1|
|     adalah|   43|
|       adam|    1|
|     adanya|    2|
|        ade|    1|
|       adek|    1|
|       adik|    6|
+-----------+-----+
only showing top 20 rows



[Stage 5:>                                                          (0 + 4) / 4]/home/student/de-prj/de-venv/lib/python3.10/site-packages/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3397
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/student/de-prj/de-venv/lib/python3.10/site-packages/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3927
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/student/de-prj/de-venv/lib/python3.10/site-packages/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3397
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/student/de-prj/de-venv/lib/python3.10/site-packages/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3927
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/student/de-prj/de-venv/lib/python3.10/site-packages/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3397
  sel

In [23]:
from malaya.stem import available_huggingface
available_huggingface

{'mesolitica/stem-lstm-512': {'Size (MB)': 35.2,
  'hidden size': 512,
  'CER': 0.02549779186652238,
  'WER': 0.05448552235248484}}